In [1]:
import gradio as gr
import transformers
import torch
import os

# Assuming DEVICE is already defined (e.g., 'cuda' or 'cpu')
DEVICE = 'cuda'  # or 'cpu' if you are not using CUDA

MODEL_DIRS = [
    # '/home/ubuntu/dev/naifu/checkopint/checkpoint-e0_s8000',
    # '/home/ubuntu/dev/naifu/checkpoint_run2/checkpoint-e0_s18000',
    # 'pixiv-prompts-gpt-finetunes/pixiv_resampled_epoch_0_batch_168178',
    # "pixiv-prompts-gpt-finetunes/pixiv_best_10m_e0_s40000",
    # 'pixiv-prompts-gpt-finetunes/8xh100_run1_e2_s26k',
    # 'pixiv-prompts-gpt-finetunes/8xh100_run2_e1_s46k',
    'pixiv-prompts-gpt-finetunes/8xh100_run2_e2_s50k',
]

# Load models and tokenizers from directories
models = {}
tokenizers = {}
for model_dir in MODEL_DIRS:
    model_name = os.path.basename(os.path.normpath(model_dir))
    models[model_name] = transformers.GPT2LMHeadModel.from_pretrained(model_dir).to(DEVICE)
    tokenizers[model_name] = transformers.AutoTokenizer.from_pretrained(model_dir)

def generate_text(rating:str, date:str, quality:str, character, prompt:str, max_length=20, selected_models=[]):
    """Generates text based on the input prompt for selected models."""

    query_prompt =f'<input rating="{rating}" chara="{character}" date="{date}" quality="{quality}" tags="{prompt}">'
    query_prompt += "<output>"

    outputs = {"original": query_prompt}
    for model_name in selected_models:
        model = models[model_name].eval()  # Set the model to evaluation mode
        tokenizer = tokenizers[model_name]
        input_ids = tokenizer.encode(query_prompt, return_tensors='pt').to(DEVICE)
        attention_mask = torch.ones(input_ids.shape, device=DEVICE)  # Create an attention mask for the inputs
        output_sequences = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_length=max_length + len(input_ids[0]),
            temperature=1.0,
            top_k=50,
            top_p=0.95,
            repetition_penalty=1.0,
            do_sample=True,
            num_return_sequences=1,
        )
        
        generated_sequence = output_sequences[0].tolist()
        text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True)
        text = text[: text.find(tokenizer.eos_token)] if tokenizer.eos_token else text  # Remove the end of sequence token

        # find the first <output> tag and remove everything before it
        text = text[text.find("<output>") + len("<output>"):].strip()
        text.replace("</output>", "")

        outputs[model_name] = text  # Store the generated text for the model

    # Return the generated texts as a dictionary
    return outputs


checkbox_choices = [os.path.basename(os.path.normpath(model_dir)) for model_dir in MODEL_DIRS]

# <input rating="sensitive" chara="" date="2020s" quality="normal" tags=""><output>
    
# Define Gradio interface
iface = gr.Interface(
    fn=generate_text,
    inputs=[
        # gr.Radio(choices=["safe", "sensitive", "nsfw", "nsfw, explicit"], label="Rating", value="safe"),
        # gr.Radio(choices=["2000s", "2010s", "2015s", "2020s"], label="Date", value="2020s"),
        # gr.Radio(choices=["bad", "normal", "good", "excellent"], label="Quality", value="excellent"),
        gr.Radio(choices=["general",  "nsfw",], label="Rating", value="safe"),
        gr.Radio(choices=["2000s", "2010s", "2020s"], label="Date", value="2020s"),
        gr.Radio(choices=["normal", "good", "excellent"], label="Quality", value="excellent"),
        gr.Textbox(lines=1, placeholder="Enter your prompt here...", label="Character"),
        gr.Textbox(lines=2, placeholder="Enter your prompt here..."),
        gr.Slider(minimum=40, maximum=300, value=140, step=10),
        gr.CheckboxGroup(choices=checkbox_choices, label="Select Models", value=checkbox_choices),
    ],
    outputs=gr.JSON(label="Generated Texts"),
    title="GPT-2 Text Generation | Multi-Model Support",
    description="This interface generates text based on the input prompt using selected models. Each model is fine-tuned from GPT-2."
)

# Launch the interface
iface.launch(share=True)


# e0_s18000: good (ip, composition)
# e0_s36000: bad (all ways)

Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://9558a639752b2a86b3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [2]:
!pip install -U pydantic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.9/407.9 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 66.3 MB/s eta 0:00:00
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.16.3
    Uninstalling pydantic_core-2.16.3:
      Successfully uninstalled pydantic_core-2.16.3
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.14
    Uninstalling pydantic-1.10.14:
      Successfully uninstalled pydantic-1.10.14


In [5]:
!pwd

/home/ubuntu/dev/prompt-upscaler-gpt2


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene